In [8]:
import requests
import pandas as pd 
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

In [9]:
def get_wb_products(query="электростимулятор", pages=3):
    all_products = []
    
    for page in range(1, pages + 1):
        url = "https://search.wb.ru/exactmatch/ru/common/v4/search"
        params = {
            "query": query,
            "resultset": "catalog",
            "limit": 100,
            "page": page,
            "appType": 1,
            "curr": "rub",
            "dest": -1257786  # Регион доставки: Москва
        }
        
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Ошибка на странице {page}")
            continue
            
        data = response.json()
        products = data.get("data", {}).get("products", [])
        
        for product in products:
            all_products.append({
                "id": product["id"],
                "name": product["name"],
                "price": product["salePriceU"] / 100,
                "rating": product.get("reviewRating", 0),
                "feedbacks": product.get("feedbacks", 0),
                "brand": product["brand"]
            })
            
    return pd.DataFrame(all_products)

# Собираем 300 товаров (3 страницы)
df = get_wb_products(pages=3)
df.to_csv("wb_products.csv", index=False)
print(f"Собрано {len(df)} товаров")

Собрано 300 товаров


In [10]:
# Инициализация драйвера один раз (вне функции)
def init_driver():
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # Фоновый режим
    chrome_options.add_argument("--disable-gpu")  # Ускоряет работу
    chrome_options.add_argument("--no-sandbox")  # Для некоторых систем
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), 
        options=chrome_options
    )
    chrome_options.add_experimental_option(
        "prefs", {"profile.managed_default_content_settings.images": 2}
    )
    return driver

# Передаём драйвер как аргумент
def get_product_details(driver, product_id):
    
    driver.get(f"https://www.wildberries.ru/catalog/{product_id}/detail.aspx")
    details = {"description": "", "power_type": "", "zones": ""}
    time.sleep(40)  # Ожидаем загрузки
    
    try:
        button_confirm_age = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/button[1]")
        button_confirm_age.click()
        time.sleep(3)
        driver.execute_script("window.scrollBy(0, 800)")
        time.sleep(20)

        # Нажатие на Характеристики
        button_characteristics_and_description = driver.find_element(By.CSS_SELECTOR, "button.product-page__btn-detail.hide-mobile.j-details-btn-desktop")
        button_characteristics_and_description.click()
        time.sleep(10)

        # Описание товара
        details["description"] = driver.find_element(By.CLASS_NAME, "option__text").text

    except Exception as e:
        print(f"Ошибка для {product_id}: {str(e)}")
    
    return details

In [4]:
driver = init_driver()


In [5]:
product_id = df.iloc[0]["id"]
details = get_product_details(driver, product_id)

In [28]:
print(details)

{'description': 'Уникальный БДСМ набор электростимуляторов 7в1\n\nЭтот эксклюзивный набор для взрослых — идеальное сочетание функциональности, качества и разнообразия для тех, кто стремится расширить свои сексуальные горизонты. Подходит как для новичков, так и для опытных пар, набор создан из безопасных, гипоаллергенных материалов, включая медицинский латекс, что гарантирует долговечность и комфорт в использовании. \n\nВ состав набора входят электростимуляторы, анально-вагинальная пробка, зажимы для сосков, электромассажеры и 4 многоразовые электроподушечки, которые можно использовать даже под одеждой. Этот набор предоставляет широкий спектр возможностей для экспериментов, помогая раскрыть новые грани чувственных наслаждений. Электростимуляция в сочетании с аксессуарами для БДСМ подарит вам и вашему партнеру невероятные ощущения и настоящий заряд эйфории.\n\nИспользуя лубриканты и гели для улучшения проводимости, вы сможете усилить эффект от стимуляции. Вибрации и электрические импульс

In [6]:
driver.quit()  # Закрываем браузер только после всех запросов

In [21]:
# Получаем характеристики и описание
def parse_wb_product(nm):
    url = f"https://card.wb.ru/cards/v2/detail?appType=1&curr=rub&dest=-1257786&spp=30&nm={nm}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        product = data["data"]["products"][0]
        print(product)
        
        return {
            "Артикул": product["id"],
            "Название": product["name"],
            # "Цена": product["salePriceU"] / 100,
            "Бренд": product["brand"],
            "Рейтинг": product["reviewRating"],
            "Отзывы": product["feedbacks"],
            # "Описание": product["description"],
            # "Характеристики": "\n".join([f"{o['name']}: {o['value']}" for o in product["options"]])
        }
        
    except Exception as e:
        print(f"Ошибка для {nm}: {str(e)}")
        return None

In [23]:
# Пример использования
data = parse_wb_product(428282572)
# print(data)

{'id': 428282572, 'root': 419796312, 'kindId': 0, 'brand': '', 'brandId': 0, 'siteBrandId': 0, 'colors': [{'name': 'белый', 'id': 16777215}, {'name': 'синий', 'id': 255}], 'subjectId': 594, 'subjectParentId': 657, 'name': 'Электростимулятор для мышц', 'entity': '', 'matchId': 0, 'supplier': 'ВсёНашлось', 'supplierId': 4533720, 'supplierRating': 4.8, 'supplierFlags': 16, 'pics': 2, 'rating': 0, 'reviewRating': 0, 'nmReviewRating': 0, 'feedbacks': 0, 'nmFeedbacks': 0, 'volume': 9, 'viewFlags': 1581058, 'isNew': True, 'promotions': [63484, 159182, 193792, 197163, 203374], 'sizes': [{'name': '', 'origName': '0', 'rank': 0, 'optionId': 609461371, 'stocks': [{'wh': 218804, 'dtype': 1, 'dist': 64, 'qty': 51, 'priority': 59023, 'time1': 60, 'time2': 46}], 'time1': 60, 'time2': 46, 'wh': 218804, 'dtype': 1, 'dist': 64, 'price': {'basic': 345000, 'product': 116400, 'total': 117300, 'logistics': 900, 'return': 0}, 'saleConditions': 134217728, 'payload': 'yOtx6BABE+yOI9WrvH2apuywUi3/AAzcBDkl9fSoRm

In [ ]:
# Получаем характеристики и описание
def parse_wb_product_1(nm):

    url = f"https://www.wildberries.ru/webapi/product/{nm}/data?subject=594&kind=0&brand=0&lang=ru"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        product = data["data"]["products"][0]
        print(product)
        
        return {
            # "Артикул": product["id"],
            # "Название": product["name"],
            # # "Цена": product["salePriceU"] / 100,
            # "Бренд": product["brand"],
            # "Рейтинг": product["reviewRating"],
            # "Отзывы": product["feedbacks"],
            # "Описание": product["description"],
            # "Характеристики": "\n".join([f"{o['name']}: {o['value']}" for o in product["options"]])
        }
        
    except Exception as e:
        print(f"Ошибка для {nm}: {str(e)}")
        return None

In [31]:
# Пример использования
data = parse_wb_product_1(428282572)

Ошибка для 428282572: 'data'


Тестыыыы

In [8]:
product_id = df.iloc[0]["id"]
driver.get(f"https://www.wildberries.ru/catalog/{product_id}/detail.aspx")

In [10]:
# Нажатие кнопки 18+
button_confirm_age = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/button[1]")
button_confirm_age.click()

In [13]:
driver.execute_script("window.scrollBy(0, 800)")

In [19]:
try:
    # Нажатие на Характеристики
    button_characteristics_and_description = driver.find_element(By.CSS_SELECTOR, "button.product-page__btn-detail.hide-mobile.j-details-btn-desktop")
    button_characteristics_and_description.click()
except Exception as e:
    print(f"Ошибка для {product_id}: {str(e)}")

# <button type="button" class="product-page__btn-detail hide-mobile j-details-btn-desktop" data-name-for-wba="Item_Description_Parameters_More" data-link="{on showDetail}text{:~t('productCard.charAndDescr')}class{merge: abCentralBlock toggle='product-page__btn-detail--ab'}" data-jsv="#2190^/2190^">Характеристики и описание</button>

In [35]:
driver.quit()  # Закрываем браузер только после всех запросов